In [1]:
# Import libraries
from dotenv import load_dotenv
import os
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display, clear_output
from sqlalchemy import create_engine

In [2]:
# Load environment variables
load_dotenv()
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_host = os.getenv('DB_HOST')
db_name = os.getenv('DB_NAME')

In [3]:
# Connect to the Stack Overflow database
conn = psycopg2.connect(
    host=db_host,
    database=db_name,
    user=db_user,
    password=db_password
)

In [4]:
# With the env variables loaded we can insert them into the engine connection string.
engine = create_engine(f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}/{db_name}")
connection = engine.connect()

In [5]:

# Define HEART metrics for Stack Overflow
heart_metrics = {
    'Happiness': 'Measure of user satisfaction and happiness with the Questions & Answers product.',
    'Engagement': 'Measure of user interaction and activity on the platform.',
    'Adoption': 'Measure of user acquisition and growth of Stack Overflow user base.',
    'Retention': 'Measure of user retention and continued usage of the platform over time.',
    'Task Success': 'Measure of user ability to accomplish their goals and tasks effectively on Stack Overflow.'
}

# Stack Overflow User Engagement Analysis

This analysis will focus on the engagement of user accounts that have been active in the last year. The decision to focus only on recently active accounts has been made to conserve time and computing resources as these accounts will provide the greatest insight into decisions made affecting SO's future. Analysis of accounts that have not been active since 2015 (for example) may be interesting, but is unlikely to be useful in the scope of this analysis. 

Engagement is defined as per the HEART metrics system which was developed by Google.

In [6]:
for key, description in heart_metrics.items():
    print(f"{key}: {description}")

Happiness: Measure of user satisfaction and happiness with the Questions & Answers product.
Engagement: Measure of user interaction and activity on the platform.
Adoption: Measure of user acquisition and growth of Stack Overflow user base.
Retention: Measure of user retention and continued usage of the platform over time.
Task Success: Measure of user ability to accomplish their goals and tasks effectively on Stack Overflow.


## Initial analysis of engaged users

In order to conduct this analysis we will need to decide what to count as an 'engaged' user. For example, we could say that a user logging in to their account would be considered engaged. Or perhaps engaged means to have posted a question? What about posting a comment? Or an answer?

For the purposes of this initial analysis we will consider an engaged user to be one that has been active in the last year and has posted either a question, answer, or comment in that time.

The first step in analysis is to pull the necessary data from the database. 

In [7]:
query = """SELECT users.id AS user_id, users.display_name,
        posts.id AS post_id, comments.id AS comment_id
FROM (
    SELECT id, display_name
    FROM users
    WHERE last_access_date >= CURRENT_DATE - interval '1 year'
) users
LEFT JOIN posts ON users.id = posts.owner_user_id
LEFT JOIN comments ON users.id = comments.user_id
"""

df = pd.read_sql(query, connection)

df

Now we can determine how many of these accounts have created at least one post.

In [ ]:
engaged_users = df[df['post_id'].notnull()]['id'].nunique()
disengaged_users = df[df['post_id'].isnull()]['id'].nunique()

print(f"Number of users with a post title: {engaged_users}")
print(f"Number of users without a post title: {disengaged_users}")

Number of users with a post title: 1080618
Number of users without a post title: 2802280
